In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [5]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [6]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [7]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([feat[:8000],x_train_text[:8000]], vad[:8000].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([feat[8000:],x_train_text[8000:]], vad[8000:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([feat[6400:],x_train_text[6400:]], batch_size=32)
# np.save('speech_text_npy', 
#          np.array(predict).reshape(3, 3639).T)


In [8]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:8000],x_train_text[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[8000:],x_train_text[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[6400:],x_train_text[6400:]], batch_size=32)
    np.save('npy8000/speech_text_npy-8000glove'+str(time)+'.npy', 
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/speech_textglove_8000.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 57s 9ms/step - loss: 1.5529 - v_loss: 0.6210 - a_loss: 0.4171 - d_loss: 0.5148 - v_ccc: 0.3790 - a_ccc: 0.5829 - d_ccc: 0.4852 - val_loss: 2.0687 - val_v_loss: 0.6792 - val_a_loss: 0.7262 - val_d_loss: 0.6633 - val_v_ccc: 0.3208 - val_a_ccc: 0.2738 - val_d_ccc: 0.3367
Epoch 2/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.1252 - v_loss: 0.3901 - a_loss: 0.3213 - d_loss: 0.4139 - v_ccc: 0.6099 - a_ccc: 0.6787 - d_ccc: 0.5861 - val_loss: 1.7658 - val_v_loss: 0.6798 - val_a_loss: 0.4966 - val_d_loss: 0.5894 - val_v_ccc: 0.3202 - val_a_ccc: 0.5034 - val_d_ccc: 0.4106
Epoch 3/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.9910 - v_loss: 0.3149 - a_loss: 0.3016 - d_loss: 0.3745 - v_ccc: 0.6851 - a_ccc: 0.6984 - d_ccc: 0.6255 - val_loss: 1.6174 - val_v_loss: 0.61

Epoch 3/50
6400/6400 [==============================] - 49s 8ms/step - loss: 0.9949 - v_loss: 0.3117 - a_loss: 0.3022 - d_loss: 0.3809 - v_ccc: 0.6883 - a_ccc: 0.6978 - d_ccc: 0.6191 - val_loss: 1.7236 - val_v_loss: 0.7675 - val_a_loss: 0.4338 - val_d_loss: 0.5222 - val_v_ccc: 0.2325 - val_a_ccc: 0.5662 - val_d_ccc: 0.4778
Epoch 4/50
6400/6400 [==============================] - 49s 8ms/step - loss: 0.8942 - v_loss: 0.2636 - a_loss: 0.2789 - d_loss: 0.3517 - v_ccc: 0.7364 - a_ccc: 0.7211 - d_ccc: 0.6483 - val_loss: 1.5131 - val_v_loss: 0.5697 - val_a_loss: 0.4248 - val_d_loss: 0.5186 - val_v_ccc: 0.4303 - val_a_ccc: 0.5752 - val_d_ccc: 0.4814
Epoch 5/50
6400/6400 [==============================] - 49s 8ms/step - loss: 0.8092 - v_loss: 0.2257 - a_loss: 0.2630 - d_loss: 0.3205 - v_ccc: 0.7743 - a_ccc: 0.7370 - d_ccc: 0.6795 - val_loss: 1.5304 - val_v_loss: 0.5670 - val_a_loss: 0.4139 - val_d_loss: 0.5495 - val_v_ccc: 0.4330 - val_a_ccc: 0.5861 - val_d_ccc: 0.4505
Epoch 6/50
6400/6400 [===

Epoch 14/50
6400/6400 [==============================] - 53s 8ms/step - loss: 0.3989 - v_loss: 0.1039 - a_loss: 0.1442 - d_loss: 0.1508 - v_ccc: 0.8961 - a_ccc: 0.8558 - d_ccc: 0.8492 - val_loss: 1.6318 - val_v_loss: 0.5669 - val_a_loss: 0.4770 - val_d_loss: 0.5878 - val_v_ccc: 0.4331 - val_a_ccc: 0.5230 - val_d_ccc: 0.4122
Epoch 15/50
6400/6400 [==============================] - 52s 8ms/step - loss: 0.3707 - v_loss: 0.1002 - a_loss: 0.1323 - d_loss: 0.1383 - v_ccc: 0.8998 - a_ccc: 0.8677 - d_ccc: 0.8617 - val_loss: 1.6254 - val_v_loss: 0.5581 - val_a_loss: 0.4880 - val_d_loss: 0.5793 - val_v_ccc: 0.4419 - val_a_ccc: 0.5120 - val_d_ccc: 0.4207
Epoch 16/50
6400/6400 [==============================] - 52s 8ms/step - loss: 0.3469 - v_loss: 0.0948 - a_loss: 0.1201 - d_loss: 0.1320 - v_ccc: 0.9052 - a_ccc: 0.8799 - d_ccc: 0.8680 - val_loss: 1.6940 - val_v_loss: 0.5629 - val_a_loss: 0.5093 - val_d_loss: 0.6218 - val_v_ccc: 0.4371 - val_a_ccc: 0.4907 - val_d_ccc: 0.3782
Epoch 17/50
2039/2039 

6400/6400 [==============================] - 54s 8ms/step - loss: 0.7440 - v_loss: 0.2001 - a_loss: 0.2476 - d_loss: 0.2963 - v_ccc: 0.7999 - a_ccc: 0.7524 - d_ccc: 0.7037 - val_loss: 1.5426 - val_v_loss: 0.5702 - val_a_loss: 0.4353 - val_d_loss: 0.5371 - val_v_ccc: 0.4298 - val_a_ccc: 0.5647 - val_d_ccc: 0.4629
Epoch 7/50
6400/6400 [==============================] - 54s 8ms/step - loss: 0.6979 - v_loss: 0.1848 - a_loss: 0.2366 - d_loss: 0.2766 - v_ccc: 0.8152 - a_ccc: 0.7635 - d_ccc: 0.7234 - val_loss: 1.5740 - val_v_loss: 0.5652 - val_a_loss: 0.4576 - val_d_loss: 0.5512 - val_v_ccc: 0.4348 - val_a_ccc: 0.5424 - val_d_ccc: 0.4488
Epoch 8/50
6400/6400 [==============================] - 54s 8ms/step - loss: 0.6382 - v_loss: 0.1639 - a_loss: 0.2186 - d_loss: 0.2557 - v_ccc: 0.8361 - a_ccc: 0.7814 - d_ccc: 0.7443 - val_loss: 1.6265 - val_v_loss: 0.5791 - val_a_loss: 0.4621 - val_d_loss: 0.5853 - val_v_ccc: 0.4209 - val_a_ccc: 0.5379 - val_d_ccc: 0.4147
Epoch 9/50
6400/6400 [==============

6400/6400 [==============================] - 63s 10ms/step - loss: 1.5448 - v_loss: 0.6162 - a_loss: 0.4191 - d_loss: 0.5095 - v_ccc: 0.3838 - a_ccc: 0.5809 - d_ccc: 0.4905 - val_loss: 2.2347 - val_v_loss: 0.7519 - val_a_loss: 0.7536 - val_d_loss: 0.7292 - val_v_ccc: 0.2481 - val_a_ccc: 0.2464 - val_d_ccc: 0.2708
Epoch 2/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.1242 - v_loss: 0.3901 - a_loss: 0.3195 - d_loss: 0.4146 - v_ccc: 0.6099 - a_ccc: 0.6805 - d_ccc: 0.5854 - val_loss: 1.6892 - val_v_loss: 0.6421 - val_a_loss: 0.4861 - val_d_loss: 0.5610 - val_v_ccc: 0.3579 - val_a_ccc: 0.5139 - val_d_ccc: 0.4390
Epoch 3/50
6400/6400 [==============================] - 55s 9ms/step - loss: 0.9819 - v_loss: 0.3061 - a_loss: 0.2982 - d_loss: 0.3777 - v_ccc: 0.6939 - a_ccc: 0.7018 - d_ccc: 0.6223 - val_loss: 1.5536 - val_v_loss: 0.5875 - val_a_loss: 0.4329 - val_d_loss: 0.5331 - val_v_ccc: 0.4125 - val_a_ccc: 0.5671 - val_d_ccc: 0.4669
Epoch 4/50
6400/6400 [=============

...

6400/6400 [==============================] - 50s 8ms/step - loss: 0.8883 - v_loss: 0.2600 - a_loss: 0.2821 - d_loss: 0.3462 - v_ccc: 0.7400 - a_ccc: 0.7179 - d_ccc: 0.6538 - val_loss: 1.6205 - val_v_loss: 0.6083 - val_a_loss: 0.4190 - val_d_loss: 0.5932 - val_v_ccc: 0.3917 - val_a_ccc: 0.5810 - val_d_ccc: 0.4068
Epoch 5/50
6400/6400 [==============================] - 49s 8ms/step - loss: 0.8110 - v_loss: 0.2260 - a_loss: 0.2650 - d_loss: 0.3199 - v_ccc: 0.7740 - a_ccc: 0.7350 - d_ccc: 0.6801 - val_loss: 1.6325 - val_v_loss: 0.5964 - val_a_loss: 0.4685 - val_d_loss: 0.5676 - val_v_ccc: 0.4036 - val_a_ccc: 0.5315 - val_d_ccc: 0.4324
Epoch 6/50
6400/6400 [==============================] - 47s 7ms/step - loss: 0.7364 - v_loss: 0.1991 - a_loss: 0.2440 - d_loss: 0.2933 - v_ccc: 0.8009 - a_ccc: 0.7560 - d_ccc: 0.7067 - val_loss: 1.6015 - val_v_loss: 0.5730 - val_a_loss: 0.4405 - val_d_loss: 0.5880 - val_v_ccc: 0.4270 - val_a_ccc: 0.5595 - val_d_ccc: 0.4120
Epoch 7/50
6400/6400 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 51s 8ms/step - loss: 0.4609 - v_loss: 0.1195 - a_loss: 0.1637 - d_loss: 0.1778 - v_ccc: 0.8805 - a_ccc: 0.8363 - d_ccc: 0.8222 - val_loss: 1.5406 - val_v_loss: 0.5400 - val_a_loss: 0.4297 - val_d_loss: 0.5709 - val_v_ccc: 0.4600 - val_a_ccc: 0.5703 - val_d_ccc: 0.4291
Epoch 13/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.4240 - v_loss: 0.1131 - a_loss: 0.1453 - d_loss: 0.1656 - v_ccc: 0.8869 - a_ccc: 0.8547 - d_ccc: 0.8344 - val_loss: 1.6297 - val_v_loss: 0.5547 - val_a_loss: 0.4955 - val_d_loss: 0.5795 - val_v_ccc: 0.4453 - val_a_ccc: 0.5045 - val_d_ccc: 0.4205
Epoch 14/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.4085 - v_loss: 0.1085 - a_loss: 0.1474 - d_loss: 0.1526 - v_ccc: 0.8915 - a_ccc: 0.8526 - d_ccc: 0.8474 - val_loss: 1.6261 - val_v_loss: 0.5539 - val_a_loss: 0.4838 - val_d_loss: 0.5884 - val_v_ccc: 0.4461 - val_a_ccc: 0.5162 - val_d_ccc: 0.4116
Epoch 15/50
6400/6400 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2039/2039 [==============================] - 5s 2ms/step
[0.4356977045536041, 0.5753230452537537, 0.5102487802505493]
19
